In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\viswa\\Documents\\MLOPs\\Smoker-Image-Classification'

In [2]:
import tensorflow as tf

In [3]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict
    mlflow_uri : str
    params_image_size : list
    params_batch_size : int

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model = "artifacts/training/model.h5",
            training_data = "artifacts/data_ingestion/Smoker-Image-classification",
            mlflow_uri = "https://dagshub.com/Viswa-Prakash/Smoker-Image-Classification.mlflow",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )

        return eval_config

In [7]:
pip install mlflow dagshub

Note: you may need to restart the kernel to use updated packages.


In [8]:
import dagshub
dagshub.init(repo_owner='Viswa-Prakash', repo_name='Smoker-Image-Classification', mlflow=True)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\viswa\anaconda3\envs\cnnsmoker\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=2fef09da-8555-485c-9543-82c0bbbb62b0&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=88fbcefc8cc670d474991cf30b97728fe09fb9ab9def30d52569698ad57d2300


[2025-06-12 22:44:29,926: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


[2025-06-12 22:44:35,660: INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/access_token "HTTP/1.1 200 OK"]
[2025-06-12 22:44:39,718: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as Viswa-Prakash

[2025-06-12 22:44:39,749: INFO: helpers: Accessing as Viswa-Prakash]
[2025-06-12 22:44:43,354: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/Viswa-Prakash/Smoker-Image-Classification "HTTP/1.1 200 OK"]
[2025-06-12 22:44:47,393: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "Viswa-Prakash/Smoker-Image-Classification"

[2025-06-12 22:44:47,435: INFO: helpers: Initialized MLflow to track repo "Viswa-Prakash/Smoker-Image-Classification"]


Repository Viswa-Prakash/Smoker-Image-Classification initialized!

[2025-06-12 22:44:47,471: INFO: helpers: Repository Viswa-Prakash/Smoker-Image-Classification initialized!]


In [9]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [15]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [16]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-06-12 22:52:13,205: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-12 22:52:13,228: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-12 22:52:13,235: INFO: common: created directory at: artifacts]


Found 214 images belonging to 2 classes.
14/14 [==============================] - 236s 17s/step - loss: 2.4155 - accuracy: 0.5607
[2025-06-12 22:56:39,349: INFO: common: json file saved at: scores.json]


2025/06/12 22:56:40 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-06-12 22:56:41,926: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\viswa\AppData\Local\Temp\tmpwandezhk\model\data\model\assets
[2025-06-12 22:56:42,706: INFO: builder_impl: Assets written to: C:\Users\viswa\AppData\Local\Temp\tmpwandezhk\model\data\model\assets]


c:\Users\viswa\anaconda3\envs\cnnsmoker\lib\site-packages\_distutils_hack\__init__.py:30: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Successfully registered model 'VGG16Model'.
2025/06/12 22:57:40 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
